# CSE 151A Group Project: Regression Model

In [1]:
# All of our imports
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

First, we must get our processed data to train the model

In [2]:
input_df = pd.read_csv('processed_input.csv')
input = input_df.to_numpy()
output_df = pd.read_csv('processed_output.csv')
output = output_df.to_numpy()
display(input_df)
display(output_df)

,host_since,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,accommodates,bedrooms,price,minimum_nights,maximum_nights,...,Bathroom essentials,Baby safety gates,Bread maker,Clothing storage,Full kitchen,Fireplace guards,Sound system,EV charger,Outdoor furniture,Lake access
0,0.735956,0.0,0.000759,1.0,0.0,0.066667,0.0,0.002761,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.577049,0.0,0.000759,1.0,1.0,0.066667,0.0,0.006653,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.523716,0.0,0.000759,1.0,0.0,0.066667,0.0,0.004852,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.573115,0.0,0.000759,1.0,1.0,0.066667,0.0,0.003052,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.493989,0.0,0.000759,1.0,0.0,0.066667,0.0,0.003168,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168318,0.467760,0.0,0.000759,1.0,1.0,0.066667,0.0,0.006653,0.000000,0.000070,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168319,0.577486,0.0,0.000759,1.0,1.0,0.066667,0.0,0.003168,0.006012,0.000164,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168320,0.704044,0.0,0.000759,1.0,1.0,0.066667,0.0,0.002587,0.005010,0.000339,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168321,0.447213,0.0,0.000759,1.0,1.0,0.066667,0.0,0.005782,0.002004,0.000199,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,review_scores_rating
0,100.0
1,100.0
2,100.0
3,100.0
4,100.0
...,...
168318,100.0
168319,100.0
168320,100.0
168321,100.0


It is important to separate our data into testing and training data, we will have a testing proportion of 0.2

In [3]:
X_train, X_test, y_train, y_test = train_test_split(input,output, test_size=0.2, random_state=42)
display(X_train.shape)
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)

(134658, 247)

(33665, 247)

(134658, 1)

(33665, 1)

Let's start off by just putting the data into a linear regression model with no other changes to the input data, we will use mean squared error as our loss function due to this being a regression problem

In [20]:
# Do basic fitting
reg = LinearRegression()
model = reg.fit(X_train,y_train)

# Show our starting off point for error
yhat_test = model.predict(X_test)
print(mean_squared_error(y_test,yhat_test))

yhat_train = model.predict(X_train)
print(mean_squared_error(y_train,yhat_train))

2.0149716574494444e+18
90.06312404052976


In [19]:
model.coef_

array([[ 2.51385393e+00,  3.58275413e+00, -1.01177403e+01,
         1.96929337e+00,  3.34746055e-01, -4.43444446e+00,
         4.75248541e+00,  1.39115502e+01,  9.65666700e-01,
        -7.27029679e+00, -1.30593837e+00, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+11,
        -1.15967304e+11, -1.15967304e+11, -1.15967304e+1